# 予測結果を可視化・提出

・horizon=16の結果

・S3のパス  
s3://work-kaggle-ohio/store-sales-time-series-forecasting/export_targetonly_noholiday_16days/


In [1]:
!pip install botocore==1.22.5

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


## エラー対応
pandasでS3パス指定でファイルを読み込むには、botocoreのバージョンが1.22.5以下である必要がある。

https://stackoverflow.com/questions/69994834/attributeerror-aioclientcreator-object-has-no-attribute-register-lazy-block

In [53]:
import boto3
import botocore

In [54]:
boto3.__version__

'1.20.23'

In [55]:
botocore.__version__

'1.22.5'

In [56]:
import pandas as pd

In [57]:
df = pd.DataFrame()
for i in range(200):
    #print(i)
    df_tmp = pd.read_csv(f's3://work-kaggle-ohio/store-sales-time-series-forecasting/export_targetonly_noholiday_16days/export_targetonly_noholiday_16days_2021-12-22T15-23-23Z_part{i}.csv')
    df = pd.concat([df, df_tmp])

In [58]:
df

,item_id,date,store_nbr,p10,p50,p90
0,automotive,2017-08-16T00:00:00Z,34,0.123840,3.054457,6.606376
1,automotive,2017-08-17T00:00:00Z,34,-0.066716,2.981440,6.437854
2,automotive,2017-08-18T00:00:00Z,34,0.079627,3.275813,7.551496
3,automotive,2017-08-19T00:00:00Z,34,0.310061,3.554194,7.638790
4,automotive,2017-08-20T00:00:00Z,34,0.079179,3.695890,7.638954
...,...,...,...,...,...,...
139,prepared foods,2017-08-27T00:00:00Z,50,68.458583,105.150105,140.948786
140,prepared foods,2017-08-28T00:00:00Z,50,54.687310,82.701220,115.284712
141,prepared foods,2017-08-29T00:00:00Z,50,41.984461,72.941851,105.144090
142,prepared foods,2017-08-30T00:00:00Z,50,49.222935,79.337880,110.063445


In [59]:
df['family'] = df['item_id'].str.upper()
df['date'] = df['date'].str[0:10]
df['sales'] = df['p50']

In [60]:
df

,item_id,date,store_nbr,p10,p50,p90,family,sales
0,automotive,2017-08-16,34,0.123840,3.054457,6.606376,AUTOMOTIVE,3.054457
1,automotive,2017-08-17,34,-0.066716,2.981440,6.437854,AUTOMOTIVE,2.981440
2,automotive,2017-08-18,34,0.079627,3.275813,7.551496,AUTOMOTIVE,3.275813
3,automotive,2017-08-19,34,0.310061,3.554194,7.638790,AUTOMOTIVE,3.554194
4,automotive,2017-08-20,34,0.079179,3.695890,7.638954,AUTOMOTIVE,3.695890
...,...,...,...,...,...,...,...,...
139,prepared foods,2017-08-27,50,68.458583,105.150105,140.948786,PREPARED FOODS,105.150105
140,prepared foods,2017-08-28,50,54.687310,82.701220,115.284712,PREPARED FOODS,82.701220
141,prepared foods,2017-08-29,50,41.984461,72.941851,105.144090,PREPARED FOODS,72.941851
142,prepared foods,2017-08-30,50,49.222935,79.337880,110.063445,PREPARED FOODS,79.337880


In [61]:
df_test = pd.read_csv('../input/test.csv')

In [62]:
df_test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


## 結合方針
・キーは、family, data, store_nbr。予測値（sales）はp50を使う。  
・その後、id, salesのみ切り出せば、提出できる。

In [63]:
df_mrg = pd.merge(df_test, df, how='left', on=['date', 'store_nbr', 'family'])

In [64]:
df_mrg[['id' ,'sales']].to_csv('../submit/sub_forecast_16days_only_target.csv', index=False)

# 完